In [ ]:
!pip install transformers

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

In [16]:
df = pd.read_csv("Articles.csv", encoding="utf-8") 
df = df.dropna()
text_data = open('Articles.txt', 'w', encoding="utf-8")
for idx, item in df.iterrows():
    article = cleaning(item["Article"])
    text_data.write(article)
text_data.close()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in pandas._libs.parsers.TextReader._convert_tokens:1083                                          │
│                                                                                                  │
│ in pandas._libs.parsers.TextReader._convert_with_dtype:1233                                      │
│                                                                                                  │
│ in pandas._libs.parsers.TextReader._string_convert:1246                                          │
│                                                                                                  │
│ in pandas._libs.parsers._string_box_utf8:1444                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 799: invalid start byte

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df = pd.read_csv("Articles.csv", encoding="utf-8")                                           │
│   2 df = df.dropna()                                                                             │
│   3 text_data = open('Articles.txt', 'w', encoding="utf-8")                                      │
│   4 for idx, item in df.iterrows():                                                              │
│                                                                                                  │
│ C:\Users\nguye\anaconda3\envs\deepwork\lib\site-packages\pandas\util\_decorators.py:211 in       │
│ wrapper                                                                                          │
│                                                                                                  │
│   208 │   │   │   │   │   raise TypeError(msg)                                                   │
│   209 │   │   │   │   else:                                                                      │
│   210 │   │   │   │   │   kwargs[new_arg_name] = new_arg_value                                   │
│ ❱ 211 │   │   │   return func(*args, **kwargs)                                                   │
│   212 │   │                                                                                      │
│   213 │   │   return cast(F, wrapper)                                                            │
│   214                                                                                            │
│                                                                                                  │
│ C:\Users\nguye\anaconda3\envs\deepwork\lib\site-packages\pandas\util\_decorators.py:331 in       │
│ wrapper                                                                                          │
│                                                                                                  │
│   328 │   │   │   │   │   FutureWarning,                                                         │
│   329 │   │   │   │   │   stacklevel=find_stack_level(),                                         │
│   330 │   │   │   │   )                                                                          │
│ ❱ 331 │   │   │   return func(*args, **kwargs)                                                   │
│   332 │   │                                                                                      │
│   333 │   │   # error: "Callable[[VarArg(Any), KwArg(Any)], Any]" has no                         │
│   334 │   │   # attribute "__signature__"                                                        │

In [9]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [11]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)
      
    model = GPT2LMHeadModel.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )
      
    trainer.train()
    trainer.save_model()

In [12]:
# you need to set parameters 
train_file_path = "/content/drive/MyDrive/Articles.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [13]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\Users\nguye\anaconda3\envs\deepwork\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 train(                                                                                      │
│    2 │   train_file_path=train_file_path,                                                        │
│    3 │   model_name=model_name,                                                                  │
│    4 │   output_dir=output_dir,                                                                  │
│                                                                                                  │
│ in train:25                                                                                      │
│                                                                                                  │
│   22 │   │     num_train_epochs,                                                                 │
│   23 │   │     save_steps):                                                                      │
│   24 │   tokenizer = GPT2Tokenizer.from_pretrained(model_name)                                   │
│ ❱ 25 │   train_dataset = load_dataset(train_file_path, tokenizer)                                │
│   26 │   data_collator = load_data_collator(tokenizer)                                           │
│   27 │                                                                                           │
│   28 │   tokenizer.save_pretrained(output_dir)                                                   │
│                                                                                                  │
│ in load_dataset:2                                                                                │
│                                                                                                  │
│    1 def load_dataset(file_path, tokenizer, block_size = 128):                                   │
│ ❱  2 │   dataset = TextDataset(                                                                  │
│    3 │   │   tokenizer = tokenizer,                                                              │
│    4 │   │   file_path = file_path,                                                              │
│    5 │   │   block_size = block_size,                                                            │
│                                                                                                  │
│ C:\Users\nguye\anaconda3\envs\deepwork\lib\site-packages\transformers\data\datasets\language_mod │
│ eling.py:61 in __init__                                                                          │
│                                                                                                  │
│    58 │   │   │   FutureWarning,                                                                 │
│    59 │   │   )                                                                                  │
│    60 │   │   if os.path.isfile(file_path) is False:                                             │
│ ❱  61 │   │   │   raise ValueError(f"Input file path {file_path} not found")                     │
│    62 │   │                                                                                      │
│    63 │   │   block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)          │
│    64                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Input file path /content/drive/MyDrive/Articles.txt not found

In [14]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [15]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "/content/drive/MyDrive/result"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
sequence = input() # oil price
max_len = int(input()) # 20
generate_text(sequence, max_len) # oil price for July June which had been low at as low as was originally stated Prices have since resumed